In [1]:
import os, re, sys
from datetime import date
import importlib
import jpnotebooks.Software.SDLC_traceability_tools.issuetracker_item_extracter2 as iie
import jpnotebooks.Software.SDLC_traceability_tools.user_tests_parser as utp
import scripts.tools.redmine_cache as rmc
from scripts.software_frs import frs_traceability2 as FRS

def reload():
    global iie, utp, rmc, UserTestsParser, RequirementExtracter
    
    iie = importlib.reload(iie)
    utp = importlib.reload(utp)
    rmc = importlib.reload(rmc)
    UserTestsParser = utp.UserTestsParser
    RequirementExtracter = iie.RequirementExtracter

reload()

In [12]:
_client = None
def get_client():
    global _client
    _client = _client or rmc.RedmineClient()
    return _client

In [33]:
from officelib.xllib import *
typnum = re.compile(r"([a-zA-Z]+)([0-9\.]+)").match

class XLRequirementsExtractor():
    def __init__(self):
        pass
    
    def extract(self, fp):
        xl = Excel()
        with screen_lock(xl):
            wb = xl.Workbooks.Open(fp)
            ws = wb.Worksheets(1)
            return self._extract(xl, wb, ws)
            
    def _extract(self, xl, wb, ws):
        cells = ws.Cells
        cr = cells.Range
        c1 = cr("A2")
        c2 = c1.End(xlc.xlDown).GetOffset(0, 4)
        data = cr(c1, c2).Value2
        out = []
        for rid, text, refs, crit, milestone in data:
            refs = refs.split()
            typ, num = typnum(rid).groups()
            ref = iie.Reference(typ, num, False, refs, text)
            ref.priority = crit
            ref.milestone = milestone
            out.append(ref)
        return out

In [34]:
basefp = "C:\\users\\nathan\\documents\\personal\\test"
ftp = "%s 190329.xlsx"
files = [ftp % typ for typ in ('URS', 'FRS', 'SWDS', 'BUG')]
x = XLRequirementsExtractor()
reqs = {}
for typ in ('URS', 'FRS', 'SWDS', 'BUG'):
    f = ftp%typ
    fp = os.path.join(basefp, f)
    reqs[typ] = x.extract(fp)

In [9]:
def newref(ref, map):
    s = ref.num.split(".", 1)
    if len(s) == 2:
        cat, num = s
        dash = "-"
    else:
        cat = s
        num = ""
        dash = ""
    return "%s-%s%s%s" % (ref.type, map[cat], dash, num)

In [ ]:
themap = {
    '1583': 'SENS',
    '2304': 'PERM',
    '2375': 'GMP',
    '3646': 'AG',
    '3651': 'TEMP',
    '3652': 'DO',
    '3653': 'pH',
    '3654': 'GAS',
    #'3655': 'COND',
    
}

In [19]:
unique = set()
for v in reqs['URS']:
#     for r in v:
        unique.add(v.num.split(".",1)[0])

issues = get_client().get_all()
for u in sorted(unique):
    print(u, issues[int(u)].subject)

1583 Sensor Selection
2304 Granular permissions
2735 Web Server Encryption
2890 Configuration File State Recorder
3646 Agitation Control
3651 Temperature
3652 DO
3653 pH
3654 Main Gas
3655 Filter Oven
3657 Pressure
3658 Level
3664 Pumps
3824 Recipes
3882 Light
3883 Power Options
3884 Alarm emails if delayed
3892 Calibration
3909 Load Bag
3910 Report Generation
3911 Display historic data
3912 Batch
3913 Integrity Test
3914 Harvest
4089 Interlocks
4090 Alarms
4094 Data Logger
4095 Reactor Name
4096 Door Control
4363 Record Retention and Protection
4364 Automated Audit Trails
4365 User Accounts and Security
4366 PID Controller Requirements
4370 Hello Interface
4374 Desktop Interface
4394 E-Stop Hardware Interlock
4399 System Variables
4400 Global Variables
4401 Web Server
4402 Embedded Controller Requirements
4404 Other Requirements
4407 Bioreactor Configuration Settings
4409 SMTP Messaging
4446 System Installers 


In [35]:
sample = [r for r in reqs['URS'] if r.num.split(".",1)[0] == '3654']

In [36]:
def q
with open("innoslate_import_test.csv", 'w') as f:
    f.write("Number,Name,Description,References,Milestone\n")
    for i, r in enumerate(sample, 1):
        num = r.tag
        name = "name %d"%i
        desc = r.text
        refs = ";".join(r.refs)
        milestone = r.milestone
        f.write("%s,%s,%s,%s,%s\n"%(num, name, desc, refs, milestone))
    

In [37]:
refs

''

In [38]:
for s in sample:
    print(s.text)

Gas Flow Control
Common
If the flow rate for an individual gas is below the minimum output, the system will "pulse" the flow of that gas such that proper average output is achieved.
Initial O2 output will be throttled until the system can detect or reasonably assume that the gas line has been purged of liquid.
If the requested percentages of individual gases is less than 100%, the system will fill the remainder of the gas mixture with Air.
If the requested percentages of individual gases exceeds 100% the system will fully satisfy requests in the following order, until 100% of flow rate is achieved: CO2, O2, N2, Air. Unfulfilled requests will be silently discarded. Gases in this list will be treated as 0% request if they are not included in main gas flow. Air will be treated as though there is always a request for 100% air.
The system will generate an alarm if actual flow rates for any individual gas differ from the requested flow rate.
MagDrive
Users will be able to manually set main g